In [5]:

import re
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from scipy import stats
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import BayesianRidge
from xgboost import XGBRegressor
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, RepeatedKFold
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
# from category_encoders import TargetEncoder
from sklearn import preprocessing
from datetime import datetime as dat
import datetime
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)



In [ ]:
train['dataset_label'] = 0
test['dataset_label'] = 1
target = 'dataset_label'

def create_adversarial_data(df_train, df_test, cols, N_val=7000):
    df_master = pd.concat([df_train[cols], df_test[cols]], axis=0)
    adversarial_val = df_master.sample(N_val, replace=False)
    adversarial_train = df_master[
        ~df_master.index.isin(adversarial_val.index)
    ]
    return adversarial_train, adversarial_val

all_cols = train.drop(["id_contract","TARGET", "id_client"] , axis='columns').select_dtypes(include=[np.number]).columns.tolist() + []

cat_cols = []
features_cols = train.drop(["id_contract","TARGET", "id_client", 'dataset_label'] , axis='columns').select_dtypes(include=[np.number]).columns.tolist() + []

adversarial_train, adversarial_test = create_adversarial_data(train, test, all_cols)

In [ ]:
train_data = Pool(
    data=adversarial_train[features_cols],
    label=adversarial_train[target],
    cat_features=cat_cols
)
holdout_data = Pool(
    data=adversarial_test[features_cols],
    label=adversarial_test[target],
    cat_features=cat_cols
)

In [ ]:
params = {
    'iterations': 100,
    'eval_metric': 'AUC',
    'od_type': 'Iter',
    'od_wait': 50,
    "depth": 2
}

model = CatBoostClassifier(**params)
_ = model.fit(train_data, eval_set=holdout_data)

In [ ]:
feature_importance =pd.DataFrame({'feature_importance': model.get_feature_importance(), 
              'feature_names': adversarial_train[features_cols].columns}).sort_values(by=['feature_importance'], 
                                                           ascending=False)
feature_importance